# Fact Qualidade

Tabela fato de inspeções de qualidade e taxas de rejeição.

In [ ]:
spark.sql(f"""
CREATE OR REPLACE TABLE {fqtn(SCHEMA_GOLD, 'fact_qualidade')} (
    inspection_id STRING,
    production_order_id STRING,
    equipment_id STRING,
    date_key INT,
    inspection_date TIMESTAMP,
    passed BOOLEAN,
    total_quantity INT,
    failed_quantity INT,
    rejection_rate_pct DOUBLE,
    defect_codes STRING
) USING DELTA
PARTITIONED BY (date_key)
""")

spark.sql(f"""
INSERT OVERWRITE {fqtn(SCHEMA_GOLD, 'fact_qualidade')}
SELECT 
    inspection_id,
    production_order_id,
    equipment_id,
    cast(date_format(inspection_date, 'yyyyMMdd') as int) as date_key,
    inspection_date,
    passed,
    total_quantity,
    failed_quantity,
    (failed_quantity / total_quantity) * 100 as rejection_rate_pct,
    defect_codes
FROM {fqtn(SCHEMA_SILVER, 'quality_inspections_clean')}
""")